In [2]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU, DepthwiseConv2D, GlobalAveragePooling2D, Dense, Reshape
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib

# --- 1. Chargement des données ---
X_train = np.load("X_train.npy")
X_test = np.load("X_test.npy")
y_train = np.load("y_train.npy")
y_test = np.load("y_test.npy")

# --- 2. Récupérer le nombre de classes ---
label_encoder = joblib.load("label_encoder.pkl")
num_classes = len(label_encoder.classes_)




c:\Users\Minfo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.6.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:
# --- 3. Mise en forme des données ---
# On reshape les MFCCs pour simuler une "image" (pour Conv2D)
# Hypothèse : MFCCs moyennés → vecteur de taille (13,) → on reshape en (13, 1, 1)
X_train = X_train.reshape(-1, 13, 1, 1)
X_test = X_test.reshape(-1, 13, 1, 1)

# One-hot encoding des labels
y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

# --- 4. Définition du modèle DS-CNN ---
def build_ds_cnn_model(input_shape, num_classes):
    inputs = Input(shape=input_shape)

    # First conv layer
    x = Conv2D(64, kernel_size=(3, 1), padding='same', activation=None)(inputs)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise separable convolutions (x3)
    for _ in range(3):
        x = DepthwiseConv2D(kernel_size=(3, 1), padding='same', activation=None)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)
        x = Conv2D(64, kernel_size=(1, 1), padding='same', activation=None)(x)
        x = BatchNormalization()(x)
        x = ReLU()(x)

    # Global average pooling + dense softmax
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# --- 5. Compilation ---
model = build_ds_cnn_model((13, 1, 1), num_classes)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
# --- 6. Entraînement ---
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_ds_cnn_model.h5", save_best_only=True)
]

history = model.fit(
    X_train, y_train_cat,
    validation_data=(X_test, y_test_cat),
    epochs=50,
    batch_size=32,
    callbacks=callbacks
)

# --- 7. Évaluation ---
loss, acc = model.evaluate(X_test, y_test_cat)
print(f"✅ Test Accuracy : {acc * 100:.2f}%")


Epoch 1/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.0000e+00 - loss: 9.3276

258/258 ━━━━━━━━━━━━━━━━━━━━ 27s 47ms/step - accuracy: 0.0000e+00 - loss: 9.3277 - val_accuracy: 0.0000e+00 - val_loss: 9.3597
Epoch 2/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 11s 42ms/step - accuracy: 0.0000e+00 - loss: 9.2101 - val_accuracy: 0.0000e+00 - val_loss: 9.5598
Epoch 3/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 4.6485e-04 - loss: 9.1437 - val_accuracy: 0.0000e+00 - val_loss: 9.9169
Epoch 4/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - accuracy: 3.0626e-04 - loss: 8.9983 - val_accuracy: 0.0000e+00 - val_loss: 10.6601
Epoch 5/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.0011 - loss: 8.8464 - val_accuracy: 0.0000e+00 - val_loss: 11.2971
Epoch 6/50
258/258 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 3.0051e-04 - loss: 8.6880 - val_accuracy: 0.0000e+00 - val_loss: 12.2292
65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - accuracy: 0.0000e+00 - loss: 9.3598
✅ Test Accuracy : 0.00%
